In [22]:
import numpy as np
import pandas as pd

In [23]:
data = pd.read_excel('assign5.xlsx')

In [24]:
data

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,1899-12-30 00:00:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,01:00:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,02:00:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,03:00:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,04:00:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2179,NaN,1900-03-28 19:00:00,4626.51840,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2180,NaN,1900-03-28 20:00:00,4567.90752,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2181,NaN,1900-03-28 21:00:00,4723.63164,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2182,NaN,1900-03-28 22:00:00,5150.27520,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
lt_3=[]
for i in range(0,data.shape[0]-3):
    lt_3.append(data['Load (kW)'][i])

In [26]:
lt_2=[]
for i in range(1,data.shape[0]-2):
    lt_2.append(data['Load (kW)'][i])

In [27]:
lt_1=[]
for i in range(2,data.shape[0]-1):
    lt_1.append(data['Load (kW)'][i])

In [28]:
lt_0=[]
for i in range(3,data.shape[0]-0):
    lt_0.append(data['Load (kW)'][i])

In [29]:
lt_3Max=np.max(lt_3)
lt_2Max=np.max(lt_2)
lt_1Max=np.max(lt_1)
lt_0Max=np.max(lt_0)
lt_3Min=np.min(lt_3)
lt_2Min=np.min(lt_2)
lt_1Min=np.min(lt_1)
lt_0Min=np.min(lt_0)
for i in range(len(lt_3)):
    lt_0[i]=(lt_0[i]-lt_0Min)/(lt_0Max-lt_0Min)
    lt_1[i]=(lt_1[i]-lt_1Min)/(lt_1Max-lt_1Min)
    lt_2[i]=(lt_2[i]-lt_2Min)/(lt_2Max-lt_2Min)
    lt_3[i]=(lt_3[i]-lt_3Min)/(lt_3Max-lt_3Min)

In [30]:
load=pd.DataFrame({'LT-3':lt_3,'LT-2':lt_2,'LT-1':lt_1, 'LT-0':lt_0})

In [31]:
from sklearn.model_selection import train_test_split
X, y = load.drop('LT-0',axis=1),load['LT-0']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 10)

In [32]:
def gradient(m1,m2,m3,c,x1,x2,x3,y):
    m1_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x1
    m2_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x2
    m3_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)*x3
    c_deriv=-1*(y-m1*x1-m2*x2-m3*x3-c)
    return m1_deriv,m2_deriv,m3_deriv,c_deriv
import random
m1=random.uniform(-1,1)
m2=random.uniform(-1,1)
m3=random.uniform(-1,1)
c=random.uniform(-1,1)
eta=0.01
max_iter=500

for iter in range(max_iter):
    for i in range(x_train.shape[0]):
        del_m1,del_m2,del_m3,del_c=gradient(m1,m2,m3,c,x_train.iat[i,0],x_train.iat[i,1],x_train.iat[i,2],y_train.iloc[i])
        m1=m1-eta*del_m1
        m2=m2-eta*del_m2
        m3=m3-eta*del_m3
        c=c-eta*del_c
print('model parameter m and c',m1,m2,m3,c)

model parameter m and c -0.2174836101464578 0.041550607301541656 0.8595017898869277 0.15383958783388538


In [33]:
y_pred_train = []
for i in range(x_train.shape[0]):
    x1=x_train.iat[i,0]
    x2=x_train.iat[i,1]
    x3=x_train.iat[i,2]
    y=y_train.iat[i]
    y_pred = (m1 * x1) + c + (m2 * x2) + (m3 * x3)
    y_pred_train.append(y_pred)
y_pred_train = np.array(y_pred_train)

In [34]:
y_pred_test = []
for i in range(x_test.shape[0]):
    x1=x_test.iat[i,0]
    x2=x_test.iat[i,1]
    x3=x_test.iat[i,2]
    y=y_test.iat[i]
    y_pred = (m1 * x1) + c + (m2 * x2) + (m3 * x3)
    y_pred_test.append(y_pred)
y_pred_test = np.array(y_pred_test)

In [35]:
y_train_denorm=[]
y_train_pred_denorm=[]
for i in range(y_train.shape[0]):
    y_train_denorm.append(y_train.iloc[i]*(lt_0Max-lt_0Min)+lt_0Min)
    y_train_pred_denorm.append(y_pred_train[i]*(lt_0Max-lt_0Min)+lt_0Min)
y_pred_train=y_train_pred_denorm
y_train=y_train_denorm

In [36]:
y_test_denorm=[]
y_test_pred_denorm=[]
for i in range(y_test.shape[0]):
    y_test_denorm.append(y_test.iloc[i]*(lt_0Max-lt_0Min)+lt_0Min)
    y_test_pred_denorm.append(y_pred_test[i]*(lt_0Max-lt_0Min)+lt_0Min)
y_pred_test=y_test_pred_denorm
y_test=y_test_denorm

In [37]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
meanSquareErr=0
root_meanSquareErr = 0
meanAbsoluteError = 0
for k in range(len(y_train)):
    # print(y_train)
    meanSquareErr += (y_train[k]-y_pred_train[k]) ** 2
    meanAbsoluteError += abs(y_train[k]-y_pred_train[k])

meanSquareErr = meanSquareErr/len(y_train)
root_meanSquareErr = math.sqrt(meanSquareErr)
meanAbsoluteError = meanAbsoluteError/len(y_train)
print('Mean square error', meanSquareErr)
print('Root mean square error', root_meanSquareErr) 
print('Mean absolute error', meanAbsoluteError)

Mean square error 415221.8389554815
Root mean square error 644.377093754489
Mean absolute error 445.7033113190417


In [38]:
meanSquareErr=0
root_meanSquareErr = 0
meanAbsoluteError = 0
for k in range(len(y_test)):
    # print(y_train)
    meanSquareErr += (y_test[k]-y_pred_test[k]) ** 2
    meanAbsoluteError += abs(y_test[k]-y_pred_test[k])

meanSquareErr = meanSquareErr/len(y_test)
root_meanSquareErr = math.sqrt(meanSquareErr)
meanAbsoluteError = meanAbsoluteError/len(y_test)
print('Mean square error', meanSquareErr)
print('Root mean square error', root_meanSquareErr) 
print('Mean absolute error', meanAbsoluteError)

Mean square error 382516.44219182746
Root mean square error 618.4791364240409
Mean absolute error 441.309139299053


In [39]:
x1 = float(input("Enter load 3 hours before :"))
x2 = float(input("Enter load 2 hours before :"))
x3 = float(input("Enter load 1 hour before :"))
NormX1=(x1-lt_3Min)/(lt_3Max-lt_3Min)
NormX2=(x2-lt_2Min)/(lt_2Max-lt_2Min)
NormX3=(x3-lt_1Min)/(lt_1Max-lt_1Min)
print('LT-3',x1)
print('LT-2',x2)
print('LT-1',x3)
pred_y=NormX1*m1+NormX2*m2+NormX3*m3+c
denormY = pred_y*(lt_0Max-lt_0Min)+lt_0Min
print("Predicted Load:",denormY)

LT-3 6000.0
LT-2 500.0
LT-1 4000.0
Predicted Load: 4063.3010281889387
